In [ ]:
import math
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt

## case 1

In [ ]:
H = 6.8
Ka = 0.14 #m/d
Kb = Ka
R = 0.001 #m/d
h = 1 #m
D = H-(h+1)
r_e = 0.10 #m
D

In [ ]:
def Hooghoudt(L_estim, D, h, r_e, R, Ka, Kb):
    
    # Auxiliary variable x (Van der Molen and Wesseling, 1991)
    x = (2*math.pi*D)/L_estim  
    
    #print('Auxiliary variable x is ', x)
    
    # Calculate equivalent depth
    D_e = (math.pi*L_estim) / ( 8 * (math.log( (L_estim)/(math.pi*r_e) ) + Vandermolen(x) ) ) 
    #print('D_e is', D_e)
    
    # Calculate L with Hooghoudt equation
    L_calc = math.sqrt( ((8*Kb*D_e*h) + (4*Ka*(h**2))) / R)
    #print('Calculated drain spacing is', L_calc )
    print(L_estim,L_calc,D,h,r_e,x)
    
    return D_e, L_calc

In [ ]:

def Vandermolen(x):
    #Fx = None   # assign a default value of None
    if x > 0.5:
        Fx = 0
        for n in range(1, 1001, 2):
            Fx += 4 * math.exp(-2 * n * x) / (n * (1 - math.exp(-2 * n * x)))
        print('Series')
        #d = (math.pi * L) / (4 * Fx + math.log(x / (2 * math.pi)))
    else:
        Fx = (((math.pi)**2)/(4*x)) + math.log(x/(2*math.pi)) 
        print('Analytic')
            #d = (math.pi * L) / (4 * Fx + math.log(x / (2 * math.pi)))
    return Fx

 

## automatic optimization

In [ ]:
def obj_fun(LL,D, h, r_e, R, Ka, Kb):
    # Calculate Hooghoudt equation and Vandemolen
    D_e, L_calc = Hooghoudt(LL, D, h, r_e, R, Ka, Kb)
    # Compare estimated and calculated L and minimize difference
    resid = (LL - L_calc)
    OF = resid**2
    
    return OF

In [ ]:
L_est=75
res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb,))
print(h)
print('The optimal drain spacing is',res.x)

In [ ]:
# Define bounds for L (optional)
from scipy.optimize import minimize

bounds = [(0, None)] # L can't be negative

# Define function to be minimized
fun = lambda L: obj_fun(L, D, h, r_e, R, Ka, Kb)

# Minimize objective function
res = minimize(fun, L_est, bounds=bounds)

# Extract optimized L value
L = res.x[0]
L

### manual optimization

In [ ]:
# Visualize relationship equivalent depth and drain spacing for this particular case
D_es=[]
LLs = np.arange(1, 200,.2)

fig, ax = plt.subplots(figsize=(6, 4))
for LL in LLs:
    D_e, L_calc= Hooghoudt(LL, D, h, r_e, R, Ka, Kb)
    D_es.append(D_e)
    ax.plot(LL,D_e,'b.')
    ax.set_xlabel('Drain spacing L (m)')
    ax.set_ylabel('Equivalent depth $D_e$ (m)')
    ax.grid('on')

# Find the index of the closest value of L to L_est
index = np.argmin(np.abs(LLs - L_est))

# Use linear interpolation to estimate D_e for L=L_est m
L1, L2 = LLs[index], LLs[index+1]
D_e1, D_e2 = D_es[index], D_es[index+1]
D_e_est = D_e1 + (L_est - L1)*(D_e2 - D_e1)/(L2 - L1)

# Print the result
print('For L='+str(L_est)+'m, D_e is approximately {:.3f}m'.format(D_e_est))    
       
# ax.plot(Lest, D_e_est, 'ro', markersize=10)
# plt.show()
D_e, L_calc= Hooghoudt(L_est, D_e_est, h, r_e, R, Ka, Kb)  
print(D_e, L_calc)
 
ax.plot(L_est, D_e_est, 'bs', markersize=10)    
tolerance = .5
diff = float('inf')

while diff > tolerance:
    # Use linear interpolation to estimate D_e for L=L_est m
    L1, L2 = LLs[index], LLs[index+1]
    D_e1, D_e2 = D_es[index], D_es[index+1]
    D_e_est = D_e1 + (L_est - L1)*(D_e2 - D_e1)/(L2 - L1)
    
    # Calculate D_e and L_calc for the current value of Lest
    D_e, L_calc = Hooghoudt(L_est, D_e_est, h, r_e, R, Ka, Kb)
    
    # Calculate the difference between Lest and L_calc
    diff = abs(L_calc - L_est)
    print(diff)
    # Update Lest based on the sign of the difference
    if L_calc > L_est:
        L_est += 0.1
    else:
        L_est -= 0.1
    #print(Lest)

# Print the final result
print('For L={:.2f}m, D_e is approximately {:.3f}m'.format(L_est, D_e))

ax.plot(L_est, D_e, 'ro', markersize=10)
plt.show()

## case 2

the same information as case 1 except a ditch with the water depth of 2.5m, bottom width of 0.5m and side slope of 1:1
is replaced by drainage pipe. The design water depth in the ditch is 0.5m. What will be the drain spacing?

In [ ]:
# calculate the wet primeter 
u = 0.5+2*math.sqrt(0.5**2+0.5**2)
r_e = u/math.pi
r_e

In [ ]:

res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
#print(h)
print('The optimal drain spacing is',res.x)

## Case 3

In [ ]:
H = 6.8
Ka = 0.06 #m/d
Kb = 0.3  #m/d
R = 0.001 #m/d
h = 1 #m
D = H-(h+1)
r_e = 0.10 #m
L_est=75

In [ ]:
res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
#print(h)
print('The optimal drain spacing is',res.x)

#### Sensitivity of spacing to Ka, Kb etc

In [ ]:
Kbs = np.arange(0.01,1.3,.05)
Ls = []
for Kb in Kas:
    res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
    #print(h)
    #print('The optimal drain spacing is',res.x)
    Ls.append(res.x)

## Ernst Equations

In [ ]:
def ernstb(Dv, Db, Dr, h, r_e, R, Ka, Kb):
    u=math.pi * r_e
    # Coefficients of the quadratic equation
    a = 1 / (8 * Kb * Db)
    b = math.log(Dr / u) / (math.pi * Kb)
    c = (Dv / Ka) - (h / R)

    # Calculate the roots of the quadratic equation
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        print("No real roots exist.")
    else:
        root1 = (-b + math.sqrt(discriminant)) / (2 * a)
        root2 = (-b - math.sqrt(discriminant)) / (2 * a)
#         print("L1 =", root1)
#         print("L2 =", root2)
    return root1, root2

In [34]:


def ernstt(alpha, Dr, Dv, Da, Db, h, r_e, R, Ka, Kb):
    # Coefficients of the quadratic equation
    a = 1 / (8*((Kb * Db)+(Ka * Da)))
    print(a,Dr)
          
    b = math.log((alpha * Dr) / (math.pi * r_e)) / (math.pi * Ka)

    c = (Dv / Ka) - (h / R)
    print(b)
    # Calculate the roots of the quadratic equation
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        print("No real roots exist.")
    else:
        root1 = (-b + math.sqrt(discriminant)) / (2 * a)
        root2 = (-b - math.sqrt(discriminant)) / (2 * a)
        print("L1 =", root1)
        print("L2 =", root2)

    
    return root1, root2

## Case 4


In [35]:
h=0.7
R = 0.007
Dr = 1
Da = Dr+.5*h
Da
alpha=3.9
Ka = 0.5
Kb = 2
Db = 4
a = 3.9
Dv = h
r_e = .05
ernstt(a, Dr, Dv, Da, Db, h, r_e, R, Ka, Kb)


0.01440922190201729 1
2.0448093021671476
L1 = 38.028764571004174
L2 = -179.93853014140421


(38.028764571004174, -179.93853014140421)

## Comparison of Hooghhoudt vs Ernst

In [ ]:
R = 0.008 #m/d
Ka = 0.4
D = 4
h = 0.8-0.4
r_e = 0.1
Kb = Ka